論文  
https://arxiv.org/abs/2103.13455<br>
<br>  
GitHub  
https://github.com/csinva/matching-with-gans<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/MatchingInGAN_Space_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## Githubからソースコードを取得

In [ ]:
%cd /content

!git clone https://github.com/csinva/matching-with-gans.git

## ライブラリのインストール

In [ ]:
!pip install scikit-learn==0.21.3 # use this version to unpickle ridge model


## ライブラリのインポート

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import sys
import matplotlib.image as mpimage
import matplotlib.pyplot as plt
import os
from os.path import join
from PIL import Image
sys.path.append('matching-with-gans')
import util
from google.colab import files

# テスト画像のセットアップ
img1<br>
https://pixabay.com/ja/photos/%e3%83%a2%e3%83%87%e3%83%ab-%e5%a5%b3-%e8%8b%a5%e3%81%84%e3%83%a2%e3%83%87%e3%83%ab-2339867/<br>
<br>
img2
<br>
https://pixabay.com/ja/photos/%e7%94%b7-%e4%ba%8b%e6%a5%ad-%e4%bb%95%e4%ba%8b-%e8%b5%b7%e6%a5%ad%e5%ae%b6-%e4%ba%ba%e9%96%93-4398724/<br>

In [ ]:
%cd /content/matching-with-gans
!rm -rf upload
!mkdir -p upload
%cd upload

temp = files.upload()
temp = list(temp.keys())
temp = temp[0]

img1_path = os.path.join("/content/matching-with-gans/upload", temp)
img1_filename = os.path.basename(img1_path)
img1_wo_ext = img1_filename.split(".")[0]
print("upload img1 file here:", img1_path)

temp = files.upload()
temp = list(temp.keys())
temp = temp[0]

img2_path = os.path.join("/content/matching-with-gans/upload", temp)
img2_filename = os.path.basename(img2_path)
img2_wo_ext = img2_filename.split(".")[0]
print("upload img2 file here:", img2_path)

In [ ]:
im = mpimage.imread(img1_path)
util.imshow(im)

## WorkSpace定義

In [ ]:
%cd /content/matching-with-gans
!rm -rf outputs
!mkdir -p outputs
%cd outputs
!mkdir aligned projected manipulated

RAW_IMAGES_DIR = "/content/matching-with-gans/upload"

ALIGNED_IMAGES_DIR = "/content/matching-with-gans/outputs/aligned"
PROJECTED_IMAGES_DIR = "/content/matching-with-gans/outputs/projected"
MANIPULATED_IMAGES_DIR = "/content/matching-with-gans/outputs/manipulated"
INTERPOLATED_IMAGES_DIR = "/content/matching-with-gans/outputs/interpolated"

# Align the images

In [ ]:
%cd /content/matching-with-gans

!python ./projection_manipulation/scripts/00_align_images.py  {RAW_IMAGES_DIR} {ALIGNED_IMAGES_DIR}

In [ ]:
im_raw = mpimage.imread(join(RAW_IMAGES_DIR, img1_filename))
im_aligned = mpimage.imread(join(ALIGNED_IMAGES_DIR, img1_wo_ext+"_01.png"))
util.plot_row([im_raw, im_aligned], annot_list=['original', 'aligned'])

# GAN潜在空間に投影

In [ ]:
%cd /content/matching-with-gans

!python ./projection_manipulation/scripts/01_project_images.py  {ALIGNED_IMAGES_DIR} {PROJECTED_IMAGES_DIR}

In [ ]:
im_aligned = mpimage.imread(join(ALIGNED_IMAGES_DIR, img1_wo_ext+"_01.png"))
im_projected = mpimage.imread(join(PROJECTED_IMAGES_DIR, img1_wo_ext+"_01.png"))
util.plot_row([im_aligned, im_projected], annot_list=['aligned', 'projected'])

# 属性変更画像生成

In [ ]:
%cd /content/matching-with-gans

ATTRS_TO_ALTER = 'ACHGMBSEW'
!python ./projection_manipulation/scripts/02_manipulate.py  {PROJECTED_IMAGES_DIR} {MANIPULATED_IMAGES_DIR} {ATTRS_TO_ALTER}

In [ ]:
Image.open(join(MANIPULATED_IMAGES_DIR, img1_wo_ext+"_01.png"))

# 2画像間の中間画像生成

In [ ]:
%cd /content/matching-with-gans

!python ./projection_manipulation/scripts/02_interpolate.py  {PROJECTED_IMAGES_DIR} {INTERPOLATED_IMAGES_DIR}

In [ ]:
Image.open(join(INTERPOLATED_IMAGES_DIR, img2_wo_ext + "_01_" + img1_wo_ext +"_01.png"))

In [ ]:
files.download('/content/matching-with-gans/outputs')